In [1]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [2]:
from rank_llm.rerank import Reranker
from rank_llm.rerank.pointwise.monot5 import MonoT5

# reranker = Reranker(MonoT5("castorini/monot5-base-med-msmarco"))
reranker = Reranker(MonoT5("castorini/monot5-3b-med-msmarco"))


/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Template validated successfully!
Successfully created pointwise inference handler!


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
from rank_llm.data import Query, Candidate, Request

def rerank_monoT5(query, doc_list):
    candidates = [Candidate(docid = i, score = 0, doc = {"segment": doc_list[i]}) for i in range(len(doc_list))]
    request = Request(query = Query(text = query, qid = 0), candidates = candidates)
    rerank_results = reranker.rerank(request, logging = True)
    reranked_doc_list = [candidate.doc["segment"] for candidate in rerank_results.candidates]
    scores = [candidate.score for candidate in rerank_results.candidates]
    return reranked_doc_list, scores

In [4]:
query = "what causes type 2 diabetes"
doc_list = ["climate change is important",
            "type 2 diabetes is caused by consuming too much sugar",
            "type 2 diabetes is a disease",
            "Type 2 diabetes is caused by a combination of insulin resistance and progressive loss of insulin production",
            "Type 2 diabetes happens when your body can’t use insulin well and can’t make enough of it to keep blood sugar normal — often due to lifestyle factors on top of genetic predisposition.",
           ]

print (rerank_monoT5(query, doc_list))

Progress through (q, d) pairs: 100%|██████████| 5/5 [00:00<00:00, 20.16it/s]

(['Type 2 diabetes is caused by a combination of insulin resistance and progressive loss of insulin production', 'type 2 diabetes is caused by consuming too much sugar', 'Type 2 diabetes happens when your body can’t use insulin well and can’t make enough of it to keep blood sugar normal — often due to lifestyle factors on top of genetic predisposition.', 'type 2 diabetes is a disease', 'climate change is important'], [0.9532461476305445, 0.9145189968345779, 0.8681682478883613, 0.008741363099235051, 4.443065585357746e-06])


In [5]:

question_71 = """A 66-year-old man is brought to the emergency room by his wife due to abdominal distension and persistent somnolence for the past 2 weeks. The patient’s wife says that he has been sleeping much more than usual for the past 5 days. His bowel and bladder habit have not changed. His past medical history is significant for alcoholic liver cirrhosis. His vital signs include: pulse 76/min, respiratory rate 15/min, temperature 38.0°C (100.4°F) and blood pressure 122/75 mm Hg. On physical examination, the patient is altered and not responsive to commands. Oriented x 0. The abdomen is significantly distended. Shifting dullness is present and a positive fluid wave can be elicited. Hyperreflexia and asterixis are noted. Laboratory findings are significant for the following:
Laboratory test
Sodium 140 mEq/L
Potassium 3.5 mEq/L
Chloride 97 mEq/L
Glucose 90 mg/dL
Ammonia 100 µg/dL (ref: 19-60 µg/dL)
Arterial blood gas
pH 7.4
pCO2 40 mm Hg
pO2 90 mm Hg
HCO3 26 mEq/L
An abdominal ultrasound shows surface nodularity compatible with cirrhosis but no other changes aside from ascites. An upper GI endoscopy is performed which shows gastric varices with no signs of active bleeding. An MRI of the brain is insignificant. What is the most likely precipitating factor that led to this patient’s condition?
"""
rag_71_1 = "[Central pontine myelinolysis--pathogenesis and review of the literature]. A 77-year-old, nonalcoholic man was admitted to the Omiya Red Cross Hospital with the complaint of fever and delirium state of two days' duration. Two months prior to admission he had had cough and sputum. Chest X-ray revealed honey comb lungs. Cultures of sputum revealed mycobacterium tuberculosis after eight weeks incubation. He had no liver disease in his past history. The patient appeared cachetic. His vital signs were as follows; temperature 38 degrees C, blood pressure 132/68 mmHg, with a pulse rate of 84/min. He was delirium and excited. Findings of the cranial, motor and sensory nerve examination were normal. Initial laboratory studies showed a serum sodium value of 133 mEq/l, potassium 4.5 mEq/l, chloride 98 mEq/l; a serum GOT value of 51 units, GPT 36 units; a total protein content of 7.8 g/dl and ESR rate of 87 mm/hr. Six days after admission, the patients' consciousness level began to stupor and nuchal rigidity was appeared. Spinal fluid examination revealed opening pressure 270 mm H2O, cell counts 720 (N 712, L 8)/cumm, sugar 57 mg/dl and protein 170 mg/dl. Spinal fluid cultures were positive in mycobacterium tuberculosis after eight weeks incubation. Laboratory studies showed a serum sodium value of 114 mEq/l, potassium 4.4 mEq/l, chloride 86 mEq/l, a serum osmorality (SeOs) 225 mOsm/l and urine osmorality (UOs) 958 mOsm/l.(ABSTRACT TRUNCATED AT 250 WORDS)"
rag_71_2 = "Acute kidney injury in patients with cirrhosis: perils and promise. A 62-year-old man with cirrhosis secondary to hepatitis C and chronic alcohol abuse was admitted to the intensive care unit with hematemesis and mental status changes. Physical examination showed ascites and stigmata of chronic liver disease. Blood pressure was noted as 87/42 mm Hg and laboratory studies showed a serum creatinine level of 0.8 mg/dL, an estimated glomerular filtration rate of 84 mL/min/1.73 m(2) calculated using the Modification of Diet in Renal Disease Study equation, a serum sodium level of 123 mEq/L, a total serum bilirubin level of 4.3 mg/dL, and an international normalization ratio of 1.6. The patient was resuscitated with packed red blood cells and fresh-frozen plasma and bleeding was controlled. However, on the third day of admission, creatinine level increased to 1.5 mg/dL. Examination of urine sediment showed 1 to 5 bilirubin-stained granular casts per high-powered field and a few renal tubular epithelial cells. The urine sodium level was 21 mEq/L and the fractional excretion of sodium was 0.43%."
rag_71_3 = "[Status epilepticus induced by prolonged immersion in hot herb bath: report of one case]. A 6-year-old girl with cerebral palsy developed conscious disturbance and generalized convulsion after one-hour hot herb drug bath. Physical examination on admission revealed rectal temperature 41 degrees C, hot skin, respiration 46/min, regular heart beat 98/min, BP 130/60 mmHg, Glascow coma scale 4 (E2M1V1), soft and flat abdomen, no hepatosplenomegaly, no skin rash, no focal neurological sign, increased generalized muscle ton. Laboratory data showed CBC: WBC 20400 cumm (Neutrophils 31%, Lymphocytes 69%), Hb 11.6gm%, ESR 11 mm/hr, arterial blood gas: PH 7.077, PO2 43mmHg, PCO2 57.1mmHg, HCO3- 16 mEq/L, BE-11.5mEq/L, serum sodium 143 mEq./L, potassium 5.2 mEq/L, chloride 101 mEq/L, free calcium ion 3.8mg%, GOT 63IU/L, GPT 263 IU/L, amylase 193 IU/L, alkaline phosphatase 388 IU/L, LDH 1245 IU/L, CPK 677 IU/L, total bilirubin 0.8 mg/dl, direct type 0.1 mg/dl, BUN 18 mg/dl, Glucose 35 mg/dl. Urinalysis revealed proteinuria( ) trace hematuria and pyuria, but no cast. Lumbar puncture is within normal limits. Bacteriology including blood and CSF are normal. Multiple organ failure was noted at that time. Intensive cooling methods were performed including central and peripheral cooling. We used luminal and valium to control the seizure. Condition didn't improve. Afterwards cardiopulmonary arrest developed. Patient expired 8 hours after admission despite of resuscitation. Heat stroke in infancy and childhood is different from that in adulthood. The predisposing factors are high ambient temperature, dehydration, very young baby, sweat gland dysfunction, or ectodermal dysplasia. Definition of heat stroke includes 1) rectal temperature above 41 degrees C, 2) behavioral change, 3) warm skin, wet or dry.(ABSTRACT TRUNCATED AT 250 WORDS)"
# ms-marco-MiniLM-L12-v2 [-5.0213623 -5.2226663 -6.552218 ]
# ms-marco-MiniLM-L6-v2 [-7.4047713 -7.993838  -7.994837 ]
# "cross-encoder/ms-marco-electra-base" [0.9226231  0.91059524 0.8697561 ]
# 'BAAI/bge-reranker-v2-m3' [-2.568312168121338, -1.3906357288360596, -6.028105735778809]
# 'BAAI/bge-reranker-v2-gemma' [3.243699550628662, 3.633350133895874, 1.231812834739685]
# ChatGPT & Claude: rag_2 is useful, 1 and 3 not useful

question_206 = "A 34-year-old woman is assaulted and suffers a number of stab wounds to her abdomen. Bystanders call paramedics and she is subsequently taken to the nearest hospital. On arrival to the emergency department, her vitals are T: 36 deg C, HR: 110 bpm, BP: 100/60, RR: 12, SaO2: 99%. A FAST and abdominal CT are promptly obtained which are demonstrated in Figures A and B, respectively. Her chart demonstrates no other medical problems and vaccinations/boosters up to date. The patient is diagnosed with a Grade V splenic laceration and is immediately brought to the OR for emergent splenectomy. The splenectomy is successfully performed with removal of the damaged spleen (Figure C). Following the operation, the patient should receive which of the following vaccines: (I) H. influenzae (II) Tetanus (III) N. meningitidis (IV) S. pneumoniae (V) Hepatitis B"
rag_206_1 = "Traumatic splenic laceration with delayed rupture secondary to coughing in a patient with Von Willebrand disease. We describe the case of a 54-year-old male with Von Willebrand Disease who presented to the Emergency Department (ED) with 2Â weeks of worsening abdominal pain after falling on his left flank while boating. On his initial presentation, he was found to have a Grade II splenic injury that was managed non operatively by the trauma service. Four days later, he returned to the ED when he developed severe abdominal pain after coughing and was found to have active extravasation from the splenic parenchyma with hemoperitoneum on CT angiography and a grossly positive FAST exam. Intraoperatively, he was found to have a Grade V splenic injury and subsequently underwent splenectomy."
rag_206_2 = "Development of Bilateral Heterotopic Ossification After Survival of Life Threatening Purpura Fulminans. Heterotopic ossification has been reported in patients who have undergone traumatic amputations, burn injuries, and total hip arthroplasty; however, the incidence of heterotopic ossification following purpura fulminans has only been reported in one case with unilateral involvement. Here we present a bilateral lower extremity case of heterotopic ossification as sequelae of purpura fulminans.Â  A 34-year-old male smoker with a past medical history of stab wounds to the chest and abdomen requiring emergent exploratory laparotomy, diaphragmatic repair, and splenectomy 15 years agoÂ presented to the emergency department with a rapid onset of high fevers, chills and myalgia. He did not receive post-splenectomy prophylactic vaccinations forÂ "
rag_206_3 = "Ectopic splenic autotransplantation following traumatic injury: A case report. A 41-year-old male patient was admitted to the General Hospital of Guangzhou Military Command due to upper abdominal pain persisting for 12 h. Computed tomography (CT) and positron emission tomography/CT scans revealed multiple soft-tissue shadows in the abdominal cavity, peritoneum and Glisson's capsule, but the metabolic activity was at normal levels. A small area of low-density shadows near the tail of the pancreas and multiple shadows of enlarged lymph nodes were identified around the porta hepatis and the pancreas, with a mildly increased metabolic activity. On the basis of the CT images the patient was diagnosed with pancreatitis. Radionuclide imaging showed the absence of the spleen from its normal position (following splenectomy), but abnormal phagocytosis of multiple red blood cells was observed in the abdomen, which was diagnosed as ectopic splenic autotransplantation (ESAT). The patient subsequently recovered well following symptomatic treatment. ESAT in trauma patients requires urgent surgery in order to remove the damaged spleen and artificially cultivate partial splenic tissue."
# ms-marco-MiniLM-L12-v2  [-8.348295  -3.5772295 -7.418079 ]
# ms-marco-MiniLM-L6-v2 [-6.214598  -1.0228055 -7.813245 ]
# "cross-encoder/ms-marco-electra-base" [0.77497405 0.8828966  0.65200865]
# 'BAAI/bge-reranker-v2-m3' [-6.462421417236328, -4.166738510131836, -5.648132801055908]
# 'BAAI/bge-reranker-v2-gemma' [4.644124984741211, 4.327882766723633, 2.2424514293670654]
# ChatGPT: 1 is useful, Claude: 2 is useful, 1 also partially useful
# test: no context wrong, 1+2+3 correct, 1 wrong, 2 correct, 3 wrong, 1+2 correct, 2+3 correct, 1+3 correct
# seems like 2 is most useful (ChatGPT reason, Claude, DeepSeek R1 all agree with this)


In [6]:
query = question_206
doc_list = ["hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world ", 
            rag_206_2, 
            "nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you ",
            "hello world",
            "nice to meet you",
            rag_206_1,
            rag_206_3,
           ]

query = question_71
doc_list = ["hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world hello world ", 
            rag_71_2, 
            "nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you nice to meet you ",
            "hello world",
            "nice to meet you",
            rag_71_1,
            rag_71_3,
           ]

reranked_doc_list, scores = rerank_monoT5(query, doc_list)
for result in zip(reranked_doc_list, scores):
    print (result[0], '\n', result[1], '\n')

Progress through (q, d) pairs: 100%|██████████| 7/7 [00:00<00:00, 11.00it/s]

Acute kidney injury in patients with cirrhosis: perils and promise. A 62-year-old man with cirrhosis secondary to hepatitis C and chronic alcohol abuse was admitted to the intensive care unit with hematemesis and mental status changes. Physical examination showed ascites and stigmata of chronic liver disease. Blood pressure was noted as 87/42 mm Hg and laboratory studies showed a serum creatinine level of 0.8 mg/dL, an estimated glomerular filtration rate of 84 mL/min/1.73 m(2) calculated using the Modification of Diet in Renal Disease Study equation, a serum sodium level of 123 mEq/L, a total serum bilirubin level of 4.3 mg/dL, and an international normalization ratio of 1.6. The patient was resuscitated with packed red blood cells and fresh-frozen plasma and bleeding was controlled. However, on the third day of admission, creatinine level increased to 1.5 mg/dL. Examination of urine sediment showed 1 to 5 bilirubin-stained granular casts per high-powered field and a few renal tubular

In [7]:

from transformers import T5ForConditionalGeneration, T5Tokenizer
model = "castorini/monot5-3b-med-msmarco"
tokenizer = T5Tokenizer.from_pretrained(model)

def get_num_tokens(prompt: str) -> int:
    return len(tokenizer.encode(prompt))

prompt = """
Query: А 43-уеаr-old mаn рrеѕеntѕ wіth tіnglіng аnd numbnеѕѕ of the lowеr lіmbѕ for 2 wееkѕ. Не also сomрlаіnѕ of реrѕіѕtеnt раіn in his legs whісh is not relieved by over-the-counter analgesics. Past medical history is significant for type 2 dіаbеtes mellitus for 2 уеаrѕ, inconsistently managed with mеtformіn аnd glіmеріrіdе. Оn physical ехаmіnаtіon, thеrе іѕ dесrеаѕеd ѕеnѕаtіon to pain in both lower lіmbs, but deep tеndon rеflехеѕ аrе іntасt. Ніѕ vіtаl ѕіgnѕ include: blood рrеѕѕurе 122/84 mm Нg, tеmреrаturе 36.7°C (98.1°F), and rеѕріrаtorу rаtе 10/mіn. His ankle-brachial pressure index (ABPI) on the right side is 1.1. His blood sugar analyses are as follows:
Fasting 141 mg/ dL
2 hours Post-Prandial 235 mg/ dL
HbA1c 8.1%
Which of the following is the best measure to prevent the progression of the symptoms present in this patient?
[A] : Use of atorvastatin
[B] : Femoro-Ileal artery bypass
[C] : Strict blood glucose control
[D] : Lower limb amputation 
Document: Hyperuricemia and the metabolic syndrome in Hangzhou. The aim of this study was to investigate prevalences of hyperuricemia and the metabolic syndrome (MS) in the Hangzhou population, and the relationship between serum uric acid and the MS. A cross-sectional study was conducted among 4155 subjects (2614 men and 1541 women) aged 20-80 years, recruited through a health check program in Hangzhou, China. Biochemical and haematological parameters were measured by standard methods. The diagnosis of the MS is made when three or four of the following criteria are met: 1) body mess index (BMI) >or= 25; 2) systolic blood pressure >or= 140 mmHg or diastolic blood pressure >or= 90 mmHg; 3) fasting triacyglycerol >or= 1.7 mmol/L (150 mg/dL), high density lipoprotein cholesterol (HDL-C)  0.9 mmol/L (35 mg/dL) in men and 1.0 mmol/L (39 mg/dL) in women; 4) fasting glucose >or= 6.1 mmol/L (109 mg/dL). Hyperuricemia is defined by cut-off values of > 420 mumol/L for men and > 360 mumol/L for women. Prevalences were 16.9% (N=702) for hyperuricemia and 8.4% (N=349) for the MS. Serum uric acid concentration was significantly higher in males than in females (p0.0001), and significantly higher in subjects with obesity, dyslipidemia and hypertension compared with those without. In the partial correlation analysis, after controlling for gender, age and creatinine, serum uric acid concentration was significantly positively correlated with BMI (r=0.301, p0.0001), systolic blood pressure (r=0.151, p0.0001), diastolic blood pressure (r=0.168, p0.0001), total cholesterol (r=0.144, p0.0001) and triacyglycerol (r=0.234, p0.0001). Results suggest that increased serum uric acid concentration is associated with an increased prevalence of metabolic disorders such Relevant: 
"""

prompt = """
[An unusual picture of insulinoma in type-2 diabetes mellitus and morbid obesity]. Marked hyperinsulinism was demonstrated in the course of an oral glucose tolerance test (oGTT) in a 63-year-old woman with severe obesity (height 1.59 m, body weight 123 kg, body-mass index 46.4 kg/m2). The diabetic metabolic state, first diagnosed 12 years ago, had been replaced by a low plasma glucose level: she often had attacks of ravenous hunger. A reducing diet of 800 kcal had not been tolerated. She had not had any syncopes. She had continually gained weight since puberty, but her weight had remained relatively constant for the past 5 years during which she had been treated with L-thyroxine for a diffuse goitre (stage II). In the course of an oGTT (75 g glucose) the basal insulin concentration (146 pmol/l) had risen to 1663 pmol/l at 30 min. The basal proinsulin level was 50 times normal (66 pmol/l vs. 1.418 pmol/l), while the initial plasma glucose level had fallen from 4.3 mmol/l to 3.8 mmol/l. Spiral computed tomography of the pancreas showed a 3 x 2.5 cm mass in the region of the tail of the pancreas. At laparoscopy a 4 cm tumor was palpated in the region of the pancreatic tail. Left resection of the pancreas was performed. Histopathological examination of the surgical specimen confirmed an insulinoma. A repeat of oGTT 6 months postoperatively demonstrated a markedly diminished insulin level compared with the preoperative results, as well as a diabetic metabolic state. In case of dramatic improvement of diabetes mellitus in an obese patient without drug treatment or weight reduction an insulin-producing tumour should be considered in the differential diagnosis. There may be no typical hypoglycaemic symptoms because of insulin resistance associated with the obesity.
"""

prompt = question_206 + "\n" + rag_206_3

print(get_num_tokens(prompt))

Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


529
